[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jmalbornoz/SimpleRAG/blob/main/1_simple_rag_example.ipynb)

# Simple RAG Example #1
## Dr José M Albornoz
### April 2024

In [ ]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/RAG/')

In [ ]:
! pip install openai langchain unstructured pdf2image tabulate libmagic num2words tiktoken ctransformers

# CHROMADB SENTENCE TRANSFORMERS

In [ ]:
#!pip install faiss-cpu
!pip install sentence-transformers
!pip install chromadb

In [ ]:
! pip freeze | grep langchain

In [ ]:
import chromadb

client = chromadb.Client()
#client.delete_collection("rag_demo")
collection = client.get_or_create_collection("rag_demo")

collection.add(
    documents=[""" "He is an ignoramus," interrupted my friend, as he stepped unsteadily
forward, while I followed immediately at his heels.  In an instant he
had reached the extremity of the niche, and finding his progress
arrested by the rock, stood stupidly bewildered.  A moment more and I
had fettered him to the granite.  In its surface were two iron staples,
distant from each other about two feet, horizontally.  From one of
these depended a short chain, from the other a padlock.  Throwing the
links about his waist, it was but the work of a few seconds to secure
it.  He was too much astounded to resist.  Withdrawing the key I
stepped back from the recess. """, "This is a document which is dummy"],
    ids=["id1", "id2"]
)

results = collection.query(
    query_texts=["What did the narrator do to his friend?"],
    n_results=1
)

In [ ]:
from pprint import pprint
context_extracted = results['documents'][0][0]
pprint(context_extracted)

 # FIRST STEP FOR RAG

# Llama-2-7B-Chat-GGML

In [ ]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 20, 'temperature': 0}
llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML', config=config)

In [ ]:
from langchain import PromptTemplate, LLMChain
template = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words from the context
<</SYS>>

Answer the question below from context below :
{context}
{question} [/INST]
"""

question_p = """What did the narrator do to his friend?"""
context_p = """ "He is an ignoramus," interrupted my friend, as he stepped unsteadily
forward, while I followed immediately at his heels.  In an instant he
had reached the extremity of the niche, and finding his progress
arrested by the rock, stood stupidly bewildered.  A moment more and I
had fettered him to the granite.  In its surface were two iron staples,
distant from each other about two feet, horizontally.  From one of
these depended a short chain, from the other a padlock.  Throwing the
links about his waist, it was but the work of a few seconds to secure
it.  He was too much astounded to resist.  Withdrawing the key I
stepped back from the recess."""

prompt = PromptTemplate(template=template, input_variables=["question","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})
response

In [ ]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 20, 'temperature': 0}
llm = CTransformers(model='TheBloke/falcon-7b-instruct-GGML', config=config)

In [ ]:
from langchain import PromptTemplate, LLMChain
template = """
User: {context}
User: {question}
Assistant:
"""

question_p = """What did the narrator do to his friend?"""
context_p = """ "He is an ignoramus," interrupted my friend, as he stepped unsteadily
forward, while I followed immediately at his heels.  In an instant he
had reached the extremity of the niche, and finding his progress
arrested by the rock, stood stupidly bewildered.  A moment more and I
had fettered him to the granite.  In its surface were two iron staples,
distant from each other about two feet, horizontally.  From one of
these depended a short chain, from the other a padlock.  Throwing the
links about his waist, it was but the work of a few seconds to secure
it.  He was too much astounded to resist.  Withdrawing the key I
stepped back from the recess."""

prompt = PromptTemplate(template=template, input_variables=["question","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})
response